# Loading Data

In [ ]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
data_loader = loader.load_data()

# Create the classifier and train it

In [11]:
import importlib
imported_module = importlib.import_module("classifier")
importlib.reload(imported_module)
import classifier
import lightning as L

model = classifier.NeuralNetwork()
trainer = L.Trainer(max_epochs=100, accelerator="auto", devices="auto", strategy="auto")
trainer.fit(model, data_loader['train'], data_loader['val'])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type    | Params | Mode 
-----------------------------------------
0 | lstm | LSTM    | 5.3 K  | train
1 | cnv  | Conv2d  | 10     | train
2 | relu | ReLU    | 0      | train
3 | loss | CTCLoss | 0      | train
-----------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

D:\Documents\GitHub\Audio-ex3\.venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

# Test the classifier

In [ ]:
trainer.test(model, data_loader['test'])